# Non-maximum Suppression
Non-maximumSuppression是一类可以让我们在一系列重叠的目标中选择最好的一个目标的算法。
## 交并比（Intersection Over Union, IoU）
IoU（有时候也叫做Jaccard系数），是我们衡量**gound truth的bbox**和**模型预测出的bbox**的重叠率的一个指标。然而在non-maximum suppression的中，我们对两个预测出的bbox计算IoU。

In [ ]:
def compute_iou(bbox_a, bbox_b):
    """Compute IoU of two bounding box 

    Args:
        bbox_a (tuple in list): [(leftupper_x, _y), (rightbottom_x, y)]
        bbox_b (same as bbox_a)

    Returns:
        [float]: IoU of two boxes
    """
    a_x1, a_y1 = bbox_a[0]
    a_x2, a_y2 = bbox_a[1]
    b_x1, b_y1 = bbox_b[0]
    b_x2, b_y2 = bbox_b[1]

    a_area = (a_x2 - a_x1) * (a_y2 - a_y1)
    b_area = (b_x2 - b_x1) * (a_y2 - a_y1)
    
    x1 = max(a_x1, b_x1)
    y1 = max(a_y1, b_y1)
    x2 = min(a_x2, b_x2)
    y2 = min(a_y2, b_y2)
    
    w = max(0, x2 - x1)
    h = max(0, y2 - y1)
    area = w * h
    
    iou  = area / (a_area + b_area - area)
    return iou

算法的输入为bbox的列表`P`，其中的元素形式为`(x1, y1, x2, y2, c)`，分别代表了bbox的左上角和右下角坐标以及此bbox的置信度（score），同时需要设置一个IoU的阈值`thresh_iou`。最后函数输出过滤后的最终bbox列表`keep`
算法分以下三步：
1. 选择`P`中置信度最高的bbox `S`，将其移入`keep`中。
2. 将`S`与`P`中剩余的bbox分别计算IoU。如果某bbox的IoU高于阈值那么从`P`中移除该bbox。
3. 如果此时`P`中仍有预测bbox，那么重复步骤一；否则返回`keep`。

In [ ]:
def NMS(P, thresh_iou:float):
    x1, y1 = P[:, 0], P[:, 1]
    x2, y2 = P[:, 2], P[:, 3]
    scores = P[:, 4]
    
    # Calculate area of every block in P
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()
    keep = list()
    
    while len(order) > 0:
        
        # Find the one with max score
        idx = order[-1]
        
        # Add it to `keep` and remove from `P` 
        keep.append(P[idx])
        order = order[:-1]
        
        if len(order) == 0:
            break
        